In [3]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
from DNN_module import Net

%run ../NCoinDP_functions.ipynb

In [4]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu"
device = torch.device(dev) 

2.1.2+cu121
True
NVIDIA A100 80GB PCIe


In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from DNN_module import Net
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

# Default : cuda
torch.set_default_device('cuda')

# Number of train and validation data
L_train = 250000
L_val   =  50000

# Define the batch size
BATCH_SIZE = 64

# Number of Epochs
N_EPOCHS = 300
    
# Number of Simulations
sims = 20

tmp = "infer_sim_1.pt"
tmp = torch.load(tmp)

# Data import
[X, Y] = tmp
X = X.to(device)
Y = Y.to(device)

# Weighted loss function
torch.set_default_device('cuda')
def weighted_mse_loss(input, target, weight):
        return (weight * (input - target) ** 2).sum()
out_range = [torch.quantile(Y, 0.01, 0).detach().cpu().numpy(), torch.quantile(Y, 0.99, 0).detach().cpu().numpy()]

weight_1 = torch.tensor(1/(out_range[1] - out_range[0])**2)
#for sim in range(0,4):
for sim in [3, 19]:
    
    torch.manual_seed(1000 + sim)
    indexes = torch.randperm(L_train + L_val)

    # Divide Data
    X_train = X[indexes[0:L_train]]
    Y_train = Y[indexes[0:L_train]]

    X_val = X[indexes[L_train:(L_train + L_val)]]
    Y_val = Y[indexes[L_train:(L_train + L_val)]]

    # Use torch.utils.data to create a DataLoader that will take care of creating batches
    dataset = TensorDataset(X_train, Y_train)
    dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, generator=torch.Generator(device=device))

    # Get the dataset size for printing (it is equal to N_SAMPLES)
    dataset_size = len(dataloader.dataset)

    # Define the input and output dimensions
    D_in, D_out = X_train.size()[1], Y_train.size()[1]

    # Create an instance of the Net class with specified dimensions
    H, H2, H3 = 256, 256, 256
    net = Net(D_in = D_in, D_out = D_out, H = H, H2 = H2, H3 = H3)

    # Model name
    model_save_name = 'infer_nets/net10/'+str(sim)+'.pt'
    path = F"./{model_save_name}"

    # The nn package also contains definitions of popular loss functions; in this case we will use Mean Squared Error (MSE) as our loss function.
    #loss_fn = torch.nn.MSELoss(reduction='sum')
    learning_rate = 1e-6
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

    train_error_plt = []
    val_error_plt = []

    torch.manual_seed(2000 + sim)

    # Loop over epochs
    for epoch in range(N_EPOCHS):
        for id_batch, (x_batch, y_batch) in enumerate(dataloader):
            y_batch_pred = net(x_batch)
            loss = weighted_mse_loss(y_batch, y_batch_pred, weight_1)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 30 ==0 and id_batch % 500 == 0:
                loss, current = loss.item(), (id_batch + 1)* len(x_batch)
                print(f"train_loss: {loss/BATCH_SIZE:>7f}  [{current:>5d}/{dataset_size:>5d}]")

        with torch.no_grad():
            net.eval()
            theta_pred_train = net(X_train)
            train_loss = weighted_mse_loss(Y_train, theta_pred_train, weight_1) / L_train
           
            train_error_plt = np.append(train_error_plt, train_loss.to("cpu"))

            theta_pred_val = net(X_val)
            #val_loss = loss_fn(Y_val, theta_pred_val) / L_val
            val_loss = weighted_mse_loss(Y_val, theta_pred_val, weight_1) / L_val
         
            val_error_plt = np.append(val_error_plt, val_loss.to("cpu"))

        if epoch % 10 ==0:
            print(f"Epoch {epoch + 1}\n-------------------------------")
            print(f"train_loss {train_loss:>7f} val_loss {val_loss:>7f}")
            print("learning rate: ", learning_rate, ", sim: ", sim)

        ## Choose Best Model
        if val_error_plt[epoch] == np.min(val_error_plt):
             best=epoch
             torch.save(net.state_dict(), path)

        if epoch % 100 ==99:
            net.load_state_dict(torch.load(path))
            learning_rate = max(learning_rate * 1e-1, 1e-9)


train_loss: 0.555986  [   64/250000]
train_loss: 0.478531  [32064/250000]
train_loss: 0.428812  [64064/250000]
train_loss: 0.435352  [96064/250000]
train_loss: 0.315381  [128064/250000]
train_loss: 0.317572  [160064/250000]
train_loss: 0.221515  [192064/250000]
train_loss: 0.200542  [224064/250000]
Epoch 1
-------------------------------
train_loss 0.279791 val_loss 0.282146
learning rate:  1e-06 , sim:  3
